Key Assumptions:
1. Use Ridge to simplify coefficients
2. Stress recent data
3. Second skellam on xg
<br>


4. give a bigger boost to miami


In [46]:
import pandas as pd
import numpy as np
import math

import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import product
from scipy.stats import skellam
from sklearn.model_selection import KFold
from datetime import datetime

In [47]:
data_raw = pd.read_csv('mls_data.csv')     # update file path to where you saved the data
data = data_raw.copy()
data = data.loc[(data['Day'].notna()) & (data['Day'] != 'Day')]
data.loc[:, 'home_goals'] = data['Score'].str.split('–').str[0].astype(int)
data.loc[:, 'away_goals'] = data['Score'].str.split('–').str[1].astype(int)


In [48]:
data

,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,home_goals,away_goals
0,Sat,2/25/2023,15:30,Nashville,1.3,2–0,0.4,NYCFC,"28,051",Geodis Park,Armando Villarreal,2,0
1,Sat,2/25/2023,19:30 (18:30),FC Cincinnati,1.7,2–1,1.4,Dynamo FC,"25,513",TQL Stadium,Chris Penso,2,1
2,Sat,2/25/2023,19:30,FC Dallas,0.9,0–1,0.8,Minnesota Utd,"19,096",Toyota Stadium,Ramy Touchan,0,1
3,Sat,2/25/2023,19:30 (18:30),Atlanta Utd,1.9,2–1,1.2,San Jose,"67,538",Mercedes-Benz Stadium,Jon Freemon,2,1
4,Sat,2/25/2023,19:30 (18:30),Philadelphia,3.2,4–1,0.6,Columbus Crew,"18,510",Subaru Park,Lukasz Szpala,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Sat,9/2/2023,19:30 (21:30),San Jose,1.7,1–1,1.8,Minnesota Utd,"16,151",PayPal Park,Alexis Da Silva,1,1
398,Sat,9/2/2023,19:30 (21:30),Seattle,1.6,2–2,1.1,Portland Timbers,"37,031",Lumen Field,Jon Freemon,2,2
399,Sun,9/3/2023,19:30 (18:30),Philadelphia,2.1,4–1,1.3,NY Red Bulls,"19,361",Subaru Park,Rubiel Vazquez,4,1
400,Sat,9/2/2023,19:30,Nashville,1.4,1–1,0.4,Charlotte,"27,902",Geodis Park,Alex Chilowicz,1,1


In [49]:
def train_and_predict(data, alpha_coeff):
    '''Train a Poisson Bradley-Terry model and produce predictions
    Args:
        data (pandas df): dataframe with cols 'Home', 'Away', 'home_goals', 'away_goals'
    Returns:
        pred (pandas df): dataframe with cols 'Home', 'Away',
            'prob_home_win', 'prob_away_win', 'prob_draw'
    '''
    goal_model_data = pd.concat(
    objs=[data[['Home','Away','home_goals']].assign(home=1).rename(
            columns={'Home':'offense', 'Away':'defense', 'home_goals':'goals'}
        ),
        data[['Away','Home','away_goals']].assign(home=0
        ).rename(columns={'Away': 'offense', 'Home':'defense', 'away_goals':'goals'})])

    poisson_model = smf.glm(formula="goals ~ home + defense + offense", data=goal_model_data,
                        family=sm.families.Poisson()).fit_regularized(alpha=alpha_coeff, L1_wt=0)
    all_teams = np.unique(data['Home'])

    pred_data = pd.DataFrame(product([1, 0], all_teams, all_teams),
        columns=['home', 'offense', 'defense']
    ).query('offense != defense' )

    pred_raw = pred_data.assign(pred_goals = poisson_model.predict(exog=pred_data))

    pred_home = pred_raw.query('home == 1').rename(
        columns={'offense':'Home', 'defense':'Away', 'pred_goals':'pred_goals_home'}
    ).loc[:, ['Home', 'Away', 'pred_goals_home']]

    pred_away = pred_raw.query('home == 0').rename(
        columns={'defense':'Home', 'offense':'Away', 'pred_goals':'pred_goals_away'}
    ).loc[:, ['Home', 'Away', 'pred_goals_away']]


    pred = pd.merge(pred_home, pred_away, on=['Home', 'Away']).assign(
        prob_home_win=lambda x: [
            sum([skellam.pmf(diff, x['pred_goals_home'][i], x['pred_goals_away'][i]) for diff in range(1, 10)]) for i in range(0, x.shape[0])
        ],
        prob_away_win=lambda x: [
            sum([skellam.pmf(diff, x['pred_goals_home'][i], x['pred_goals_away'][i]) for diff in range(-1, -10, -1)]) for i in range(0, x.shape[0])
        ],
        prob_draw=lambda x: [
            skellam.pmf(0, x['pred_goals_home'][i], x['pred_goals_away'][i]) for i in range(0, x.shape[0])
        ]
        ).loc[:, ['Home', 'Away', 'prob_home_win', 'prob_away_win', 'prob_draw']]

    return(pred)

Cross-Validation

In [50]:
def columnselect(row):
    if row['home_goals'] > row['away_goals']:
        return row['prob_home_win']
    elif row['home_goals'] < row['away_goals']:
        return row['prob_away_win']
    else:
        return row['prob_draw']

In [51]:
for alpha_coeff in np.arange(0.0, 0.3, 0.1):
  kf = KFold(n_splits=10, shuffle=True, random_state=42)
  cv_data = pd.DataFrame()

  for train_idx, test_idx in kf.split(data):
      pred = train_and_predict(data.iloc[train_idx], alpha_coeff)
      test = data.iloc[test_idx]

      # Calculate the log of the predicted probability for the outcome that occurred
      cv_data_k = pd.merge(test, pred, on = ['Home', 'Away'], how='left')
      cv_data_k['prob'] = cv_data_k.apply(columnselect, axis=1)
      cv_data_k['log_prob'] = cv_data_k['prob'].apply(math.log)
      cv_data_k = cv_data_k[['Date', 'Home', 'Away', 'log_prob']]
      cv_data = pd.concat(objs=[cv_data, cv_data_k])

  print(alpha_coeff, ': ', np.mean(cv_data['log_prob']))
  # Found max likelihood at alpha=0.1

0.0 :  -1.0707500953491171


KeyboardInterrupt: ignored

However, this is wrong validation; we need this season's predictions. <br>
Thus we should oversample this season's data and validate on various folds of this season's data.

In [52]:
date_format = '%m/%d/%Y'
data['Date'] = data['Date'].apply(lambda x: datetime.strptime(x, date_format).date())

In [53]:
data

,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,home_goals,away_goals
0,Sat,2023-02-25,15:30,Nashville,1.3,2–0,0.4,NYCFC,"28,051",Geodis Park,Armando Villarreal,2,0
1,Sat,2023-02-25,19:30 (18:30),FC Cincinnati,1.7,2–1,1.4,Dynamo FC,"25,513",TQL Stadium,Chris Penso,2,1
2,Sat,2023-02-25,19:30,FC Dallas,0.9,0–1,0.8,Minnesota Utd,"19,096",Toyota Stadium,Ramy Touchan,0,1
3,Sat,2023-02-25,19:30 (18:30),Atlanta Utd,1.9,2–1,1.2,San Jose,"67,538",Mercedes-Benz Stadium,Jon Freemon,2,1
4,Sat,2023-02-25,19:30 (18:30),Philadelphia,3.2,4–1,0.6,Columbus Crew,"18,510",Subaru Park,Lukasz Szpala,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,Sat,2023-09-02,19:30 (21:30),San Jose,1.7,1–1,1.8,Minnesota Utd,"16,151",PayPal Park,Alexis Da Silva,1,1
398,Sat,2023-09-02,19:30 (21:30),Seattle,1.6,2–2,1.1,Portland Timbers,"37,031",Lumen Field,Jon Freemon,2,2
399,Sun,2023-09-03,19:30 (18:30),Philadelphia,2.1,4–1,1.3,NY Red Bulls,"19,361",Subaru Park,Rubiel Vazquez,4,1
400,Sat,2023-09-02,19:30,Nashville,1.4,1–1,0.4,Charlotte,"27,902",Geodis Park,Alex Chilowicz,1,1


In [54]:
date0802 = datetime.strptime('08/02/2023', date_format).date()
date0901 = datetime.strptime('09/01/2023', date_format).date()
recent_traindata = data.loc[(data['Date']>date0802) & (data['Date']<date0901)]
real_pred = train_and_predict(data.loc[data['Date']<date0901], 0.1)
real_test = data.loc[data['Date']>=date0901]

# Calculate the log of the predicted probability for the outcome that occurred
real_cv_data = pd.merge(real_test, real_pred, on = ['Home', 'Away'], how='left')

real_cv_data['prob'] = real_cv_data.apply(columnselect, axis=1)
real_cv_data['log_prob'] = real_cv_data['prob'].apply(math.log)
real_cv_data = real_cv_data[['Date', 'Home', 'Away', 'log_prob']]
print(np.mean(real_cv_data['log_prob']))
real_cv_data

-1.1870498145421884


,Date,Home,Away,log_prob
0,2023-09-02,NYCFC,Vancouver,-1.364313
1,2023-09-02,CF Montréal,Columbus Crew,-1.163403
2,2023-09-02,D.C. United,Chicago Fire,-0.684512
3,2023-09-02,FC Cincinnati,Orlando City,-1.424038
4,2023-09-02,New England,Austin,-1.482334
5,2023-09-02,FC Dallas,Atlanta Utd,-1.347933
6,2023-09-02,LA Galaxy,Dynamo FC,-1.369964
7,2023-09-02,Sporting KC,St. Louis,-0.802892
8,2023-09-02,Real Salt Lake,Colorado Rapids,-0.619717
9,2023-09-02,San Jose,Minnesota Utd,-1.356371


In [55]:
for rep in range(0, 1000, 200):
  real_pred = train_and_predict(pd.concat([recent_traindata] * rep + [data.loc[data['Date']<date0901]]), 0.1)
  real_test = data.loc[data['Date']>=date0901]

  # Calculate the log of the predicted probability for the outcome that occurred
  real_cv_data = pd.merge(real_test, real_pred, on = ['Home', 'Away'], how='left')

  real_cv_data['prob'] = real_cv_data.apply(columnselect, axis=1)
  real_cv_data['log_prob'] = real_cv_data['prob'].apply(math.log)
  real_cv_data = real_cv_data[['Date', 'Home', 'Away', 'log_prob']]
  print(np.mean(real_cv_data['log_prob']))


-1.1870498145421884
-1.1304763328083152
-1.128910813896333
-1.1283700390472644
-1.1280959668958581


In [68]:
# Drop Inter Miami's outlier draw
data = data.drop(382)


In [73]:
future_traindata = data.loc[(data['Date']>date0802)]
future_pred = train_and_predict(pd.concat([future_traindata] * 70 + [data]), 0.1)

future_pred.to_csv('pred_mls.csv', index=False)

The following is for NWSL.

In [80]:
data_raw = pd.read_csv('nwsl_data.csv')     # update file path to where you saved the data
data = data_raw.copy()
data = data.loc[(data['Day'].notna()) & (data['Day'] != 'Day')]
data.loc[:, 'home_goals'] = data['Score'].str.split('–').str[0].astype(int)
data.loc[:, 'away_goals'] = data['Score'].str.split('–').str[1].astype(int)


In [81]:
def train_and_predict(data, alpha_coeff):
    '''Train a Poisson Bradley-Terry model and produce predictions
    Args:
        data (pandas df): dataframe with cols 'Home', 'Away', 'home_goals', 'away_goals'
    Returns:
        pred (pandas df): dataframe with cols 'Home', 'Away',
            'prob_home_win', 'prob_away_win', 'prob_draw'
    '''
    goal_model_data = pd.concat(
    objs=[data[['Home','Away','home_goals']].assign(home=1).rename(
            columns={'Home':'offense', 'Away':'defense', 'home_goals':'goals'}
        ),
        data[['Away','Home','away_goals']].assign(home=0
        ).rename(columns={'Away': 'offense', 'Home':'defense', 'away_goals':'goals'})])

    poisson_model = smf.glm(formula="goals ~ home + defense + offense", data=goal_model_data,
                        family=sm.families.Poisson()).fit_regularized(alpha=alpha_coeff, L1_wt=0)
    all_teams = np.unique(data['Home'])

    pred_data = pd.DataFrame(product([1, 0], all_teams, all_teams),
        columns=['home', 'offense', 'defense']
    ).query('offense != defense' )

    pred_raw = pred_data.assign(pred_goals = poisson_model.predict(exog=pred_data))

    pred_home = pred_raw.query('home == 1').rename(
        columns={'offense':'Home', 'defense':'Away', 'pred_goals':'pred_goals_home'}
    ).loc[:, ['Home', 'Away', 'pred_goals_home']]

    pred_away = pred_raw.query('home == 0').rename(
        columns={'defense':'Home', 'offense':'Away', 'pred_goals':'pred_goals_away'}
    ).loc[:, ['Home', 'Away', 'pred_goals_away']]


    pred = pd.merge(pred_home, pred_away, on=['Home', 'Away']).assign(
        prob_home_win=lambda x: [
            sum([skellam.pmf(diff, x['pred_goals_home'][i], x['pred_goals_away'][i]) for diff in range(1, 10)]) for i in range(0, x.shape[0])
        ],
        prob_away_win=lambda x: [
            sum([skellam.pmf(diff, x['pred_goals_home'][i], x['pred_goals_away'][i]) for diff in range(-1, -10, -1)]) for i in range(0, x.shape[0])
        ],
        prob_draw=lambda x: [
            skellam.pmf(0, x['pred_goals_home'][i], x['pred_goals_away'][i]) for i in range(0, x.shape[0])
        ]
        ).loc[:, ['Home', 'Away', 'prob_home_win', 'prob_away_win', 'prob_draw']]

    return(pred)

Cross-Validation

In [82]:
def columnselect(row):
    if row['home_goals'] > row['away_goals']:
        return row['prob_home_win']
    elif row['home_goals'] < row['away_goals']:
        return row['prob_away_win']
    else:
        return row['prob_draw']

In [83]:
for alpha_coeff in np.arange(0.5, 1.31, 0.4):
  kf = KFold(n_splits=10, shuffle=True, random_state=42)
  cv_data = pd.DataFrame()

  for train_idx, test_idx in kf.split(data):
      pred = train_and_predict(data.iloc[train_idx], alpha_coeff)
      test = data.iloc[test_idx]

      # Calculate the log of the predicted probability for the outcome that occurred
      cv_data_k = pd.merge(test, pred, on = ['Home', 'Away'], how='left')
      # print(cv_data_k)
      # cv_data_k = test
      cv_data_k['prob'] = cv_data_k.apply(columnselect, axis=1)
      cv_data_k['log_prob'] = cv_data_k['prob'].apply(math.log)
      cv_data_k = cv_data_k[['Date', 'Home', 'Away', 'log_prob']]
      cv_data = pd.concat(objs=[cv_data, cv_data_k])

  print(alpha_coeff, ': ', np.mean(cv_data['log_prob']))
  # Found max likelihood at alpha=0.9

0.5 :  -1.083856515025924
0.9 :  -1.0830541954797792
1.3 :  -1.083384302580239


However, this is wrong validation; we need this season's predictions. <br>
Thus we should oversample this season's data and validate on various folds of this season's data.

In [84]:
date_format = '%m/%d/%Y'
data['Date'] = data['Date'].apply(lambda x: datetime.strptime(x, date_format).date())

In [85]:
date0601 = datetime.strptime('06/01/2023', date_format).date()
date0901 = datetime.strptime('09/01/2023', date_format).date()
recent_traindata = data.loc[(data['Date']>date0601) & (data['Date']<date0901)]
real_pred = train_and_predict(data.loc[data['Date']<date0901], 0.9)
real_test = data.loc[data['Date']>=date0901]

# Calculate the log of the predicted probability for the outcome that occurred
real_cv_data = pd.merge(real_test, real_pred, on = ['Home', 'Away'], how='left')

real_cv_data['prob'] = real_cv_data.apply(columnselect, axis=1)
real_cv_data['log_prob'] = real_cv_data['prob'].apply(math.log)
real_cv_data = real_cv_data[['Date', 'Home', 'Away', 'log_prob']]
print(np.mean(real_cv_data['log_prob']))
real_cv_data

-1.056121897154467


,Date,Home,Away,log_prob
0,2023-09-01,Current,Angel City,-1.059279
1,2023-09-02,Courage,Gotham FC,-1.262945
2,2023-09-02,Louisville,Thorns,-0.985010
3,2023-09-03,Reign,Pride,-0.929282
4,2023-09-03,Spirit,Red Stars,-1.166799
5,2023-09-03,Wave,Dash,-0.933416


In [86]:
for rep in range(0, 500, 50):
  real_pred = train_and_predict(pd.concat([recent_traindata] * rep + [data.loc[data['Date']<date0901]]), 0.9)
  real_test = data.loc[data['Date']>=date0901]

  # Calculate the log of the predicted probability for the outcome that occurred
  real_cv_data = pd.merge(real_test, real_pred, on = ['Home', 'Away'], how='left')

  real_cv_data['prob'] = real_cv_data.apply(columnselect, axis=1)
  real_cv_data['log_prob'] = real_cv_data['prob'].apply(math.log)
  real_cv_data = real_cv_data[['Date', 'Home', 'Away', 'log_prob']]
  print(np.mean(real_cv_data['log_prob']))


-1.056121897154467
-1.074327674814515
-1.074728594714088
-1.0748661793852574
-1.0749357359557823
-1.0749777177142086
-1.0750058095758974
-1.075025926384541
-1.0750410420809142
-1.0750528154207721


In [88]:
future_traindata = data.loc[(data['Date']>date0601)]
future_pred = train_and_predict(pd.concat([future_traindata] * 70 + [data]), 0.1)

future_pred.to_csv('pred_nwsl.csv', index=False)